# Model

In [42]:
import numpy as np
import os

from glob import glob
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers

In [18]:
DATA_DIR = os.path.join('..', 'data')
DATASET_DIR = os.path.join(DATA_DIR, 'datasets', '1')

WIDTH = 300
HEIGHT = 300
CLASSES = 3

In [29]:
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
VAL_DIR = os.path.join(DATASET_DIR, 'test')

train_files = glob(os.path.join(TRAIN_DIR, '*.jpg'))
val_files = glob(os.path.join(VAL_DIR, '*.jpg'))

n_train_samples = len(train_files)
n_val_samples = len(val_files)

n_train_samples, n_val_samples

(39905, 9977)

In [30]:
model = ResNet50(weights='imagenet',
                 include_top=False,
                 input_shape=(WIDTH, HEIGHT, 3))

In [31]:
TRAINABLE_LAYERS = 30
FC_SIZE = 1024
DROPOUT = 0.5

In [32]:
x = model.output
x = Flatten()(x)
x = Dense(FC_SIZE, activation='relu')(x)
x = Dropout(DROPOUT)(x)
predictions = Dense(CLASSES, activation='sigmoid')(x)

In [33]:
LR = 0.0001
MOMENTUM = 0.9

In [34]:
model_final = Model(inputs=model.input, outputs=predictions)

model_final.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=LR, momentum=MOMENTUM),
                    metrics=['accuracy'])

## Data augmentation

In [35]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [36]:
BATCH_SIZE = 80

In [59]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    fill_mode="nearest",
    zoom_range=0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=30)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [58]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    classes=[''],
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    classes=[''],    
    target_size=(HEIGHT, WIDTH),
    class_mode="categorical")

Found 39905 images belonging to 1 classes.
Found 9977 images belonging to 1 classes.


In [65]:
def build_data_generator(gen):
    x = next(gen)
    import pdb; pdb.set_trace()
    yield x

In [ ]:
x = build_data_generator(train_generator)
next(x)

> <ipython-input-65-7fd013c6d99a>(4)build_data_generator()
-> yield x
(Pdb) x
(array([[[[ -80.2658    ,  -98.18749   , -105.0068    ],
         [ -80.478485  , -100.70269   , -107.29579   ],
         [ -85.68585   , -102.30907   , -112.583496  ],
         ...,
         [ -88.939     , -100.779     , -111.68      ],
         [ -88.939     , -100.779     , -111.68      ],
         [ -88.77158   , -100.61158   , -111.51258   ]],

        [[ -77.86174   ,  -95.18242   , -102.60274   ],
         [ -81.981026  , -101.60421   , -107.896805  ],
         [ -83.2818    , -101.40755   , -110.47995   ],
         ...,
         [ -87.09389   ,  -98.93388   , -109.834885  ],
         [ -85.39189   ,  -97.23189   , -108.13289   ],
         [ -83.839355  ,  -95.67935   , -106.58035   ]],

        [[ -75.99917   ,  -92.718834  , -100.619835  ],
         [ -83.48356   , -102.50574   , -108.497826  ],
         [ -80.87774   , -100.50603   , -108.376396  ],
         ...,
         [ -83.16828   ,  -95.00828

## Training

In [40]:
EPOCHS = 30

In [47]:
checkpoint = ModelCheckpoint("checkpoint.h5",
    monitor = 'val_acc',
    verbose = 1,
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto',
    period = 1)

early = EarlyStopping(
    monitor = 'val_acc',
    min_delta = 0,
    patience = 10,
    verbose = 1,
    mode = 'auto')

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    patience = 5,
    min_lr = 0.001)

history = model_final.fit_generator(
    train_generator,
    steps_per_epoch = n_train_samples // BATCH_SIZE,
    epochs = EPOCHS, 
    validation_data = validation_generator,
    validation_steps = n_val_samples // BATCH_SIZE,
    callbacks = [checkpoint, early, reduce_lr])

model_final.save('model.h5')

Epoch 1/30


StopIteration: 